In [1]:
import pandas as pd
import timeit
import numpy as np

from sqlalchemy import create_engine
import psycopg2
from config import db_password

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials # if possible no login, then ideal

cid ="639041944adb4c779ee9e271ddab14d3" # personal
secret = "ddec38f1f44345c4b235e8c38f27fa4c"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
genre_list = ['opera','ska','world']


In [28]:
prep = [1, 141,[]]

egs=[]
for i, t in enumerate(prep):
    if t == []:
        t = np.nan
    egs.append(t)

egs

[1, 141, nan]

In [ ]:
# # use 1 id, make album/artist search API call to test if statement
# for genre in genre_list:
#     # save track-type search in variable
#     track_looped = sp.search(q='genre:'+genre, type='track',limit=1) # set limit to number of songs to grab per genre.
#     time.sleep(90)        

#     # loop through saved search results, append according to their index/position within the nested levels of dictionaries/lists
#     for i, t in enumerate(track_looped['tracks']['items']):
#         artist_name.append(t['artists'][0]['name'])
#         artist_id.append(t['artists'][0]['id'])
#         album_name.append(t['album']['name'])
#         track_name.append(t['name'])
#         track_id.append(t['id'])
#         popularity.append(t['popularity'])
#         artist_genre.append(genre)

In [ ]:
# Start timer
start = timeit.default_timer()

# Set empty lists for each feature
track_name = []
popularity = []
track_id = []
artist_name = []
artist_id = []
artist_genre = []
album_name = []
album_id = []
release_date = []
followers = []

# Loop through the genre list and use each genre as a parameter when running the track-type search 
for genre in genre_list:
# save track-type search in variable
    track_looped = sp.search(q='genre:'+genre, type='track',limit=50) # set limit to number of songs to grab per genre
# loop through saved search results, append according to their index/position within the nested levels of dictionaries/lists

for i, t in enumerate(track_looped['tracks']['items']):
    artist_name.append(t['artists'][0]['name'])
    artist_id.append(t['artists'][0]['id'])
    album_name.append(t['album']['name'])
    track_name.append(t['name'])
    track_id.append(t['id'])
    popularity.append(t['popularity'])
    artist_genre.append(genre)

# Using the album names collected from the first search, conduct album-type search to retrieve release date (only found with album search)
for album in album_name:
    album_looped = sp.search(q='album:'+album, type='album',limit=1)
    # similarly, append appropriate metrics found within the nests
    for i, a in enumerate(album_looped['albums']['items']):
        album_id.append(a['id'])
        release_date.append(a['release_date'])

# Lastly, using the artist names collected from first search, conduct artist-type search to retrieve number of followers per artist (only found with artist search)
for name in artist_name: 
    artist_looped = sp.search(q='artist:'+name, type='artist',limit=1)
    for i, n in enumerate(artist_looped['artists']['items']):
        followers.append(n['followers']['total'])

stop = timeit.default_timer()
print(f'This took {stop - start} seconds to complete.')

In [2]:
# Track data
# Searches can be refined to q='genre:[list of genres]'
track_results = sp.search(q='year:2021',type='track',limit=15)
tr = track_results['tracks']['items'][0]

tr_name = tr['name']
tr_id = tr['id']
tr_popularity = tr['popularity']
tr_uri = tr['uri']
tr_artist_name = tr['artists'][0]['name']
tr_artist_id = tr['artists'][0]['id']
tr_album_name = tr['album']['name']
tr_album_id = tr['album']['id']

print(f"Track name: {tr_name}")
print(f"Spotify track ID: {tr_id}")
print(f"Popularity score: {tr_popularity}")
print(f"Track URI: {tr_uri}")
print(f"Main Artist: {tr_artist_name}")
print(f"Main Artist ID: {tr_artist_id}")
print(f"Album name: {tr_album_name}")
print(f"Album ID: {tr_album_id}")

Track name: We Don't Talk About Bruno
Spotify track ID: 52xJxFP6TqMuO4Yt0eOkMz
Popularity score: 95
Track URI: spotify:track:52xJxFP6TqMuO4Yt0eOkMz
Main Artist: Carolina Gaitán - La Gaita
Main Artist ID: 29PgYEggDV3cDP9QYTogwv
Album name: Encanto (Original Motion Picture Soundtrack)
Album ID: 25L8ck3KGcmCo3901ztPzR


In [14]:
track_results

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=year%3A2021&type=track&offset=0&limit=15',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4aXXDj9aZnlshx7mzj3W1N'},
       'href': 'https://api.spotify.com/v1/artists/4aXXDj9aZnlshx7mzj3W1N',
       'id': '4aXXDj9aZnlshx7mzj3W1N',
       'name': 'Lin-Manuel Miranda',
       'type': 'artist',
       'uri': 'spotify:artist:4aXXDj9aZnlshx7mzj3W1N'},
      {'external_urls': {'spotify': 'https://open.spotify.com/artist/5NTEjLv3GEX1Vx0ITx9Pxe'},
       'href': 'https://api.spotify.com/v1/artists/5NTEjLv3GEX1Vx0ITx9Pxe',
       'id': '5NTEjLv3GEX1Vx0ITx9Pxe',
       'name': 'Germaine Franco',
       'type': 'artist',
       'uri': 'spotify:artist:5NTEjLv3GEX1Vx0ITx9Pxe'},
      {'external_urls': {'spotify': 'https://open.spotify.com/artist/3xLU748QxpTmIVaiNXXg0P'},
       'href': 'https://api.spotify.com/v1/artists/3xLU748QxpTmIVaiNXXg0P',
       'id': '3

In [3]:
# Use the album name from track search to get Album data including album release date
album_results = sp.search(q='album:' +tr_album_name, type='album', limit=15)
a = album_results['albums']['items'][0]

a_name = a['name']
a_id = a['id']
a_release_date = a['release_date']
# a_genre = a['genres']

print(f"Album Name: {a_name}")
print(f"Spotify Album ID: {a_id}")
print(f"Album release date: {a_release_date}")
# print(f"Album genres: {a_genre}")

Album Name: Encanto (Original Motion Picture Soundtrack)
Spotify Album ID: 25L8ck3KGcmCo3901ztPzR
Album release date: 2021-11-19


In [4]:
# Similarly, get Artist data by querying artist name from track search
artist_results = sp.search(q='artist:'+tr_artist_name, type='artist', limit=15)
r = artist_results['artists']['items'][0]

r_followers = r['followers']['total']
r_genre = r['genres']

print(f"Artist name (first artist): {r['name']}")
print(f"Spotify Artist ID: {r['id']}")
print(f"Number of artist followers: {r_followers}")
print(f"Genres: {r_genre}")


Artist name (first artist): Carolina Gaitán - La Gaita
Spotify Artist ID: 29PgYEggDV3cDP9QYTogwv
Number of artist followers: 22875
Genres: []


In [5]:
# empty lists where results will be stored
track_name = []
popularity = []
track_id = []
track_uri = []
artist_name = []
artist_id = []
genre = []
album_name = []
album_id = []
release_date = []
followers = []

start = timeit.default_timer()

for i in range(0, 15, 50):
    track_looped = sp.search(q='year:2021', type='track', limit=15, offset=i)
    for i, t in enumerate(track_looped['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        artist_id.append(t['artists'][0]['id'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])
        track_uri.append(t['uri'])

for i in range(0, 15, 50):
    album_looped = sp.search(q='year:2021', type='album', limit=15, offset=i)
    for i, a in enumerate(album_looped['albums']['items']):
        album_name.append(a['name'])
        album_id.append(a['id'])
        release_date.append(a['release_date'])

artist_looped = []
for name in artist_name:
    
    artist_looped.append(sp.search(q='artist:'+name, type='artist', limit=15, offset=enumerate(artist_name)))

for i, f in enumerate(artist_looped):
    followers.append(f['artists']['items'][0]['followers']['total'])
    try:
        genre.append(f['artists']['items'][0]['genres']) # genres returns a list of associated genres, if using 1 genre add [0] to ['genres']
    except:
        genre.append(np.nan)
    
stop = timeit.default_timer()
print (f'This code took {stop - start} seconds to complete.')

This code took 1.8535440999999997 seconds to complete.


In [6]:
followers

[22875,
 377,
 34841381,
 224895,
 7947578,
 8099,
 229831,
 26952,
 62190603,
 224895,
 15162823,
 33,
 355953,
 26952,
 39069457]

In [7]:
genre # append genre if in preapproved genre list
# how do we get enough non-NaN genre songs


[[],
 [],
 ['permanent wave', 'pop'],
 ['art rock',
  'folk rock',
  'mellow gold',
  'new wave pop',
  'power pop',
  'pub rock',
  'rock',
  'roots rock'],
 ['florida rap', 'hip hop', 'melodic rap', 'miami hip hop', 'rap', 'trap'],
 ['bubblegum pop', 'folk rock'],
 ['boom bap', 'buffalo hip hop', 'hip hop', 'rap', 'upstate ny rap'],
 ['huayno peruano', 'tunantada'],
 ['canadian hip hop', 'canadian pop', 'hip hop', 'rap', 'toronto rap'],
 ['art rock',
  'folk rock',
  'mellow gold',
  'new wave pop',
  'power pop',
  'pub rock',
  'rock',
  'roots rock'],
 ['pop'],
 [],
 ['modern alternative pop'],
 ['huayno peruano', 'tunantada'],
 ['modern rock', 'rock']]

In [8]:
approved_genres = ['rock','rap','pop']
final_list = []
for song in genre:
    if song in approved_genres:
        final_list.append(song)
print(final_list)

[]


In [9]:
f"genre: {len(genre)} followers: {len(followers)}"

'genre: 15 followers: 15'

In [10]:
# create DataFrame
artist_df = pd.DataFrame({
    'artist_name':artist_name, 
    'artist_id':artist_id,
    'followers':followers
    })

album_df = pd.DataFrame({
    'album_name':album_name,
    'album_id':album_id,
    'release_date':release_date
})

track_features_df = pd.DataFrame({
    'track_name':track_name, 
    'track_id':track_id, 
    'track_uri':track_uri,
    'popularity':popularity,
    'genre':genre
})

In [11]:
artist_df

,artist_name,artist_id,followers
0,Carolina Gaitán - La Gaita,29PgYEggDV3cDP9QYTogwv,22875
1,LYMEY,6dPOSlnP4zgYg7bkAkp4dY,377
2,Coldplay,4gzpq5DPGxSnKTe4SA8HAU,34841381
3,Elvis Costello & The Attractions,4qmHkMxr6pTWh5Zo74odpH,224895
4,Kodak Black,46SHBwWsqBkxI7EeeBEQG7,7947578
5,Zager & Evans,55IyYO6fmYpYw6Nd4YF7bw,8099
6,Benny The Butcher,5Matrg5du62bXwer29cU5T,229831
7,Amaranta,18MbBAbtk2Knvmk75JXFqd,26952
8,Drake,3TVXtAsR1Inumwj472S9r4,62190603
9,Elvis Costello & The Attractions,4qmHkMxr6pTWh5Zo74odpH,224895


In [12]:
album_df

,album_name,album_id,release_date
0,Encanto (Original Motion Picture Soundtrack),25L8ck3KGcmCo3901ztPzR,2021-11-19
1,Compilation of the Year 2021,3qyF7adpUWWAu15eNjqS5r,2021-12-03
2,Sorry 4 The Wait,6MVsBi4m5eAzfjb6C3JKwp,2022-01-14
3,This Year's Model (2021 Remaster),4RLIesiAVONV4fOUlOSmr4,1978-03-17
4,SOUR,6s84u2TUpR3wdUv4NgKA2j,2021-05-21
5,Year 2021,1OmbkKynA01LEr5swjpKD4,2021-01-04
6,Still Over It,4lPqFAvgmG97pxyxQsyCQx,2021-11-05
7,New Year 2021,11jK3WjJ04zh6wr4Ra8GnD,2020-10-23
8,Certified Lover Boy,3SpBlxme9WbeQdI9kx7KAV,2021-09-03
9,A State Of Trance Year Mix 2021 (Mixed by Armi...,4OaMsk8VihZmMNIx8oYdoo,2021-12-09


In [13]:
track_features_df

,track_name,track_id,track_uri,popularity,genre
0,We Don't Talk About Bruno,52xJxFP6TqMuO4Yt0eOkMz,spotify:track:52xJxFP6TqMuO4Yt0eOkMz,95,[]
1,END OF YEAR 2021,6usFCFN0F69JcAaHtB55sg,spotify:track:6usFCFN0F69JcAaHtB55sg,28,[]
2,My Universe,3FeVmId7tL5YN8B7R3imoM,spotify:track:3FeVmId7tL5YN8B7R3imoM,92,"[permanent wave, pop]"
3,Pump It Up - 2021 Remaster,3oyc1mIdCBGaU55wX7otqM,spotify:track:3oyc1mIdCBGaU55wX7otqM,66,"[art rock, folk rock, mellow gold, new wave po..."
4,Super Gremlin,4A8cWXxKfIL3lAyUDzXbCF,spotify:track:4A8cWXxKfIL3lAyUDzXbCF,90,"[florida rap, hip hop, melodic rap, miami hip ..."
5,In the Year 2525,3LdLGNKCW0dNr14JQlZPQt,spotify:track:3LdLGNKCW0dNr14JQlZPQt,41,"[bubblegum pop, folk rock]"
6,Johnny P's Caddy,6AshXllQhobwSXsdpgp41w,spotify:track:6AshXllQhobwSXsdpgp41w,79,"[boom bap, buffalo hip hop, hip hop, rap, upst..."
7,Medley Picaflor de Los Andes - Live Streaming ...,5PraiQQAIkl0xw6pSFVX5h,spotify:track:5PraiQQAIkl0xw6pSFVX5h,27,"[huayno peruano, tunantada]"
8,Knife Talk (with 21 Savage ft. Project Pat),2BcMwX1MPV6ZHP4tUT9uq6,spotify:track:2BcMwX1MPV6ZHP4tUT9uq6,90,"[canadian hip hop, canadian pop, hip hop, rap,..."
9,"Radio, Radio - 2021 Remaster",0qvo5kciIqZJn1t8jdfqDn,spotify:track:0qvo5kciIqZJn1t8jdfqDn,50,"[art rock, folk rock, mellow gold, new wave po..."
